# Traveling Ethiopia Search Problem Question #5

In [1]:
# Install ROS Noetic (if not already installed or if runtime resets)
# This setup is for an Ubuntu environment, which Colab provides.

# 1. Add ROS apt repository (only if not already added)
# The `lsb_release -sc` command gets the Ubuntu codename (e.g., 'focal' for Noetic)
!sudo sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'
# 2. Add public key
!sudo apt install curl -y # Ensure curl is installed
!curl -s https://raw.githubusercontent.com/ros/rosdistro/master/ros.asc | sudo apt-key add -
# 3. Update apt package list
!sudo apt update
# 4. Install ROS Noetic Base (includes rospy, geometry_msgs, nav_msgs, tf.transformations)
!sudo apt install -y ros-noetic-ros-base

# IMPORTANT: Sourcing the setup script (!source /opt/ros/noetic/setup.bash)
# in a notebook cell only affects that specific cell's shell environment.
# For Python scripts to find ROS modules, Python's path needs to include ROS's site-packages.
# This is usually handled by a full ROS environment setup or by specific environment variables
# like PYTHONPATH. Installing `ros-noetic-ros-base` *should* put the python libraries
# in a location where the system's python can find them, but sometimes manual intervention is needed.

import math
import time
# Try-except block for rospy import to handle cases where installation might not fully integrate
_ROS_MODULES_LOADED_SUCCESSFULLY = False # Module-level flag
try:
    import rospy
    from geometry_msgs.msg import Twist, Point
    from nav_msgs.msg import Odometry
    from tf.transformations import euler_from_quaternion
    _ROS_MODULES_LOADED_SUCCESSFULLY = True
    print("ROS modules imported successfully. Note: A running ROS master is still required for full functionality.")
except ModuleNotFoundError:
    print("WARNING: ROS modules (rospy, geometry_msgs, nav_msgs, tf.transformations) could not be imported.")
    print("This usually means ROS is not installed or its environment is not correctly sourced.")
    print("Proceeding with ROS-dependent functionality disabled.")

from collections import deque

class EthiopiaNavigator:
    def __init__(self):
        self.ros_available = _ROS_MODULES_LOADED_SUCCESSFULLY # Initialize instance flag

        # 1. DEFINE MAP (Adjacency List from Figure 5)
        self.graph = {
            'Addis Ababa': ['Ambo', 'Adama', 'Debre Birhan'],
            'Ambo': ['Addis Ababa', 'Wolkite', 'Nekemte'],
            'Adama': ['Addis Ababa', 'Matahara', 'Asella', 'Batu'],
            'Debre Birhan': ['Addis Ababa', 'Debre Sina'],
            'Matahara': ['Adama', 'Awash'],
            'Awash': ['Matahara', 'Chiro', 'Gabi Rasu'],
            'Chiro': ['Awash', 'Dire Dawa'],
            'Dire Dawa': ['Chiro', 'Harar'],
            'Harar': ['Dire Dawa', 'Babile'],
            'Babile': ['Harar', 'Jigjiga'],
            'Jigjiga': ['Babile', 'Dega Habur'],
            'Dega Habur': ['Jigjiga', 'Kebri Dehar'],
            'Kebri Dehar': ['Dega Habur', 'Gode', 'Werder'],
            'Gode': ['Kebri Dehar', 'Dollo', 'Mokadisho'],
            'Nekemte': ['Ambo', 'Bedelle', 'Gimbi'],
            'Gimbi': ['Nekemte', 'Dembi Dollo'],
            'Dembi Dollo': ['Gimbi', 'Gambella', 'Assosa'],
            'Gambella': ['Dembi Dollo', 'Gore'],
            'Gore': ['Gambella', 'Tepi', 'Bedelle'],
            'Tepi': ['Gore', 'Bonga', 'Mezan Teferi'],
            'Bonga': ['Tepi', 'Jimma', 'Dawro'],
            'Jimma': ['Bonga', 'Wolkite', 'Bedelle'],
            'Wolkite': ['Ambo', 'Worabe', 'Jimma'],
            'Worabe': ['Wolkite', 'Buta Jirra', 'Hossana'],
            'Buta Jirra': ['Worabe', 'Batu'],
            'Batu': ['Adama', 'Buta Jirra', 'Shashemene'],
            'Shashemene': ['Batu', 'Hossana', 'Hawassa', 'Dodola'],
            'Hossana': ['Worabe', 'Shashemene', 'Wolaita Sodo'],
            'Wolaita Sodo': ['Hossana', 'Dawro', 'Arba Minch'],
            'Arba Minch': ['Wolaita Sodo', 'Konso', 'Basketo'],
            'Hawassa': ['Shashemene', 'Dilla'],
            'Dilla': ['Hawassa', 'Bule Hora'],
            'Dodola': ['Shashemene', 'Assasa', 'Bale'],
            'Assasa': ['Dodola', 'Asella'],
            'Asella': ['Assasa', 'Adama'],
            'Bale': ['Dodola', 'Goba', 'Sof Oumer', 'Liben'],
            'Sof Oumer': ['Bale', 'Goba', 'Gode'],
            'Goba': ['Bale', 'Sof Oumer'],
            # ... (Assume remaining connections follow previous figures)
        }

        # 2. DEFINE COORDINATES (Cartesian mapping of Figure 5)
        # Addis Ababa is origin (0,0). Scale is arbitrary for simulation.e
        self.coordinates = {
            'Addis Ababa': (0, 0),
            'Ambo': (-3, 0),
            'Adama': (3, -1),
            'Debre Birhan': (1, 3),
            'Nekemte': (-6, 1),
            'Gimbi': (-9, 2),
            'Wolkite': (-3, -3),
            'Bedelle': (-6, -1),
            'Jimma': (-6, -3),
            'Batu': (2, -4),
            'Asella': (4, -3),
            'Matahara': (6, -0.5),
            'Awash': (8, 0),
            'Chiro': (10, 1),
            'Dire Dawa': (12, 2),
            'Harar': (14, 1),
            'Babile': (16, 0.5),
            'Jigjiga': (18, 0),
            'Shashemene': (2, -6),
            'Hawassa': (1, -8),
            'Hossana': (-1, -5),
            'Wolaita Sodo': (-2, -7),
            'Arba Minch': (-3, -9),
            # ... Add others as needed
        }

        if self.ros_available: # Use instance variable
            # ROS Setup
            try:
                rospy.init_node('ethiopia_traveler', anonymous=True)
                self.vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
                self.pose_sub = rospy.Subscriber('/odom', Odometry, self.odom_callback)
                self.current_pose = Point()
                self.current_yaw = 0.0
                print("ROS node initialized and publishers/subscribers set up.")
            except rospy.exceptions.ROSInitException as e:
                print(f"WARNING: Could not initialize ROS node: {e}")
                print("This usually happens if a ROS Master (roscore) is not running.")
                print("ROS-dependent functionality will be disabled.")
                self.ros_available = False # Update instance variable
                # Define placeholders for ROS objects if ROS is not available
                self.current_pose = type('Point', (object,), {'x': 0.0, 'y': 0.0, 'z': 0.0})() # Mock Point object
                self.current_yaw = 0.0
        else:
            print("ROS functionality is disabled due to missing modules or failed initialization.")
            # Define placeholders for ROS objects if ROS is not available
            self.current_pose = type('Point', (object,), {'x': 0.0, 'y': 0.0, 'z': 0.0})() # Mock Point object
            self.current_yaw = 0.0


    def odom_callback(self, msg):
        if self.ros_available: # Use instance variable
            self.current_pose = msg.pose.pose.position
            rot = msg.pose.pose.orientation
            (_, _, self.current_yaw) = euler_from_quaternion([rot.x, rot.y, rot.z, rot.w])
        else:
            pass # No ROS, no callback


    def bfs_search(self, start, goal):
        """Uninformed Search Strategy: Breadth-First Search"""
        queue = deque([[start]])
        visited = set([start])

        while queue:
            path = queue.popleft()
            node = path[-1]

            if node == goal:
                return path

            for neighbor in self.graph.get(node, []):
                if neighbor not in visited:
                    visited.add(neighbor)
                    new_path = list(path)
                    new_path.append(neighbor)
                    queue.append(new_path)
        return None

    def move_to_goal(self, target_city):
        """Simple P-Controller to drive robot to x,y coordinate"""
        if target_city not in self.coordinates:
            print(f"Coordinates for {target_city} not defined.")
            return

        target_x, target_y = self.coordinates[target_city]
        print(f"Navigating to {target_city} at ({target_x}, {target_y})...")

        if self.ros_available: # Use instance variable
            rate = rospy.Rate(10)
            while not rospy.is_shutdown():
                dx = target_x - self.current_pose.x
                dy = target_y - self.current_pose.y
                distance = math.sqrt(dx**2 + dy**2)

                # Goal reached threshold
                if distance < 0.5:
                    print(f"Arrived at {target_city}!")
                    self.vel_pub.publish(Twist()) # Stop
                    break

                target_angle = math.atan2(dy, dx)
                angle_error = target_angle - self.current_yaw

                # Normalize angle
                if angle_error > math.pi: angle_error -= 2*math.pi
                if angle_error < -math.pi: angle_error += 2*math.pi

                cmd = Twist()
                # Simple Proportional Control
                if abs(angle_error) > 0.1:
                    cmd.angular.z = 0.5 * angle_error
                    cmd.linear.x = 0.0
                else:
                    cmd.linear.x = 0.5 * distance # Slow down as we get closer
                    if cmd.linear.x > 0.5: cmd.linear.x = 0.5 # Max speed cap
                    cmd.angular.z = 0.0

                self.vel_pub.publish(cmd)
                rate.sleep()
        else:
            print(f"Simulating arrival at {target_city}. ROS movement commands are disabled.")
            time.sleep(2) # Simulate travel time

    def execute_mission(self, start_city, goal_city):
        path = self.bfs_search(start_city, goal_city)

        if not path:
            print("No path found!")
            return

        print(f"Path Generated: {path}")

        # Teleport to start (conceptually) or assume starting there
        # Then drive through the path
        for city in path:
            # Skip start node if we are already there, or just drive to all
            self.move_to_goal(city)
            time.sleep(1) # Pause at city

# --- MAIN EXECUTION BLOCK ---
if __name__ == '__main__':
    try:
        # Example: Travel from Addis Ababa to Harar
        bot = EthiopiaNavigator()
        # Give time for connections if ROS is active, otherwise just a delay
        time.sleep(1)

        bot.execute_mission('Addis Ababa', 'Harar')

    except Exception as e: # Catch generic exceptions if ROSInterruptException is not defined yet or other errors occur
        print(f"An error occurred during mission execution: {e}")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.21).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Ign:6 http://packages.ros.org/ros/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Err:9 http://packages.ros.org/ros/ubuntu jammy Release
  404  Not Found [IP: 64.50.236.52 80]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Pack